In [5]:
import gzip
import math
import numpy
import random
import sklearn
import string
from collections import defaultdict
from nltk.stem.porter import *
from sklearn import linear_model
from gensim.models import Word2Vec
import dateutil
from scipy.sparse import lil_matrix # To build sparse feature matrices, if you like

In [6]:
answers = {}

In [7]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [8]:
### Question 1

In [17]:
dataset = []

f = gzip.open("steam_category.json.gz")
for l in f:
    d = eval(l)
    dataset.append(d)
    if len(dataset) >= 20000:
        break
        
f.close()

In [18]:
Ntrain = 10000
Ntest = 10000

dataTrain = dataset[:Ntrain]
dataTest = dataset[Ntrain:Ntrain + Ntest]

In [19]:
sp = set(string.punctuation)

In [ ]:
cntWord = defaultdict(int) #referenced from 11/14 lecture 
totalWords = 0
for d in dataTrain:
    r = d['text'].lower()
    r = ''.join([c for c in r if c not in sp])
    ws = r.split()
    for w in ws:
        cntWord[w] += 1
        totalWords += 1

totalWords

710018

In [35]:
len(cntWord)

29692

In [59]:
counts = [(v, k) for k, v in cntWord.items()]
counts.sort(reverse=True)


In [60]:
counts

[(34211, 'the'),
 (19392, 'and'),
 (18791, 'a'),
 (18077, 'to'),
 (15043, 'game'),
 (14095, 'of'),
 (13000, 'is'),
 (12735, 'you'),
 (12204, 'i'),
 (11824, 'it'),
 (9548, 'this'),
 (8274, 'in'),
 (7060, 'that'),
 (6526, 'for'),
 (6321, 'but'),
 (5586, 'with'),
 (5144, 'its'),
 (4849, 'are'),
 (4559, 'on'),
 (4291, 'as'),
 (4042, 'have'),
 (3955, 'not'),
 (3571, 'if'),
 (3540, 'like'),
 (3486, 'be'),
 (3165, 'can'),
 (3098, 'your'),
 (3006, 'so'),
 (2925, 'was'),
 (2872, 'just'),
 (2782, 'all'),
 (2764, 'or'),
 (2727, 'good'),
 (2591, 'one'),
 (2571, 'at'),
 (2564, 'more'),
 (2525, 'get'),
 (2447, 'play'),
 (2357, 'my'),
 (2309, 'there'),
 (2281, 'an'),
 (2280, 'games'),
 (2265, 'some'),
 (2253, 'fun'),
 (2251, 'really'),
 (2141, 'from'),
 (2123, 'very'),
 (2014, 'time'),
 (1985, 'will'),
 (1924, 'they'),
 (1917, 'me'),
 (1896, 'has'),
 (1882, 'out'),
 (1882, 'great'),
 (1881, 'up'),
 (1806, 'no'),
 (1751, 'story'),
 (1725, 'what'),
 (1701, 'even'),
 (1614, 'dont'),
 (1611, 'only'),
 (1

In [61]:
answers['Q1'] = counts[:10]

In [62]:
assertFloatList([x[0] for x in answers['Q1']], 10)

In [63]:
### Question 2

In [64]:
NW = 1000 # dictionary size

In [65]:
words = [x[1] for x in counts[:NW]]

In [ ]:
words

In [68]:
wordID = dict(zip(words, range(NW)))
wordSet = set(words)

In [76]:
def feature(datum): #referenced from 11/14 lecture 
    pos = [0] * NW
    r = datum['text'].lower()
    r = ''.join([c for c in r if c not in sp])
    ws = r.split()
    for w in ws:
        if w in wordSet:
            pos[wordID[w]] += 1

    return pos + [1]

In [108]:
# Build X...

X = [feature(d) for d in dataset]
y = [d['genreID'] for d in dataset]

In [109]:
Xtrain = X[:Ntrain]
ytrain = y[:Ntrain]
Xtest = X[Ntrain:Ntrain + Ntest]
ytest = y[Ntrain:Ntrain + Ntest]

In [112]:
mod = linear_model.LogisticRegression(C=1)

In [115]:
mod.fit(Xtrain, ytrain)
correct = list()
for i in range(Ntest):
    if mod.predict([Xtest[i]]) == ytest[i]:
        correct.append(1)
    else:
        correct.append(0)

c:\Users\Seraph\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [116]:
answers['Q2'] = sum(correct) / len(correct)

In [118]:
assertFloat(answers['Q2'])

In [ ]:
### Question 3

In [149]:
def tf(word, d=dataTrain[0]):
    r = d['text'].lower()
    r = ''.join([c for c in r if c not in sp])
    ws = r.split()
    #print(ws)
    return ws.count(word)

def idf(word, documents):
    count = 0
    for d in documents:
        r = d['text'].lower()
        r = ''.join([c for c in r if c not in sp])
        ws = r.split()
        if word in ws:
            count += 1
    if not count: return 0
    return math.log(len(documents) / count, 10)


In [150]:
words = ['character', 'game', 'length', 'a', 'it']

ans = list()
for word in words:
    idf_ = idf(word, dataTrain)
    tfidf = tf(word) * idf_
    ans.append((idf_, tfidf))


In [151]:
answers['Q3'] = ans

In [152]:
assertFloatList([x[0] for x in answers['Q3']], 5)
assertFloatList([x[1] for x in answers['Q3']], 5)

In [ ]:
### Question 4

In [156]:
idfs = defaultdict(float)
for word in wordSet:
    idfs[word] = idf(word, dataTrain)

In [164]:
def feature(datum):
    pos = [0] * NW
    r = datum['text'].lower()
    r = ''.join([c for c in r if c not in sp])
    ws = r.split()
    for w in ws:
        if w in wordSet:
            pos[wordID[w]] = tf(w, datum) * idfs[w]

    return pos + [1]

In [165]:
X = [feature(d) for d in dataset]
y = [d['genreID'] for d in dataset]

In [166]:
Xtrain = X[:Ntrain]
ytrain = y[:Ntrain]
Xtest = X[Ntrain:]
ytest = y[Ntrain:]

In [167]:
mod = linear_model.LogisticRegression(C=1)

In [168]:
mod.fit(Xtrain, ytrain)
correct = list()
for i in range(Ntest):
    if mod.predict([Xtest[i]]) == ytest[i]:
        correct.append(1)
    else:
        correct.append(0)

c:\Users\Seraph\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [169]:
answers['Q4'] = sum(correct) / len(correct)

In [171]:
assertFloat(answers['Q4'])

In [ ]:
### Question 5

In [180]:
def Cosine(x1,x2):
    dot = numpy.dot(x1, x2)
    norm1 = numpy.linalg.norm(x1)
    norm2 = numpy.linalg.norm(x2)
    if norm1 == 0 or norm2 == 0:
        return 0
    return dot / (norm1 * norm2)

In [181]:
similarities = list()
trainTFIDF = list()
for _, word in counts:
    trainTFIDF.append(tf(word) * idfs[word])


In [182]:
for d in dataTest:
    testTFIDF = list()
    for _, word in counts:
        testTFIDF.append(tf(word, d) * idfs[word])
    similarities.append(Cosine(trainTFIDF, testTFIDF))

KeyboardInterrupt: 

In [ ]:
similarities.sort(reverse=True)

In [ ]:
answers['Q5'] = similarities[0]

In [ ]:
assertFloat(answers['Q5'][0])

In [ ]:
### Question 6

In [ ]:
answers['Q6'] = 

In [ ]:
assertFloat(answers['Q6'])

In [ ]:
### Question 7

In [ ]:
import dateutil.parser

In [ ]:
dataset = []

f = gzip.open("young_adult_20000.json.gz")
for l in f:
    d = eval(l)
    d['datetime'] = dateutil.parser.parse(d['date_added'])
    dataset.append(d)
    if len(dataset) >= 20000:
        break
        
f.close()

In [ ]:
model5 = Word2Vec(reviewLists,
                  min_count=1, # Words/items with fewer instances are discarded
                  vector_size=5, # Model dimensionality
                  window=3, # Window size
                  sg=1) # Skip-gram model

In [ ]:
answers['Q7'] = res[:5]

In [ ]:
assertFloatList([x[1] for x in answers['Q7']], 5)

In [172]:
f = open("answers_hw4.txt", 'w')
f.write(str(answers) + '\n')
f.close()